# This notebook is to make some processing of the audios transcribed

The transcriptions of lectures and conferences contain a lot of nonsensical speech.
- Repetitions of words and fragments of phrases
- Incomplete sentences, interactions with the public.

All of these speech forms do not add meaning to the chunks and their embeddings.

***The goal of this notebook is to process it a bit with GPT*** 

The input will be 

.json files with chunks 

# We will use ollama

See (https://www.postman.com/postman-student-programs/workspace/ollama-api/documentation/21521806-f48dc31a-a9f1-4dad-9082-fd07f5cd2fda)

We will have two api endpoints :
- /api/generate -> for single prompt generation
- /api/chat -> for chat conversations

## Important!!! Check the models that DO have Chat 

- llama2 models offer chat
- mistral and mixtral do not!


# setup stuff


In [35]:
import requests
import json

# Define the ollama URL
ollama_url = "http://localhost:11434"
ollama_generate_url = ollama_url+"/api/generate"
ollama_chat_url = ollama_url+"/api/chat"
# Choose your prefered model
ollama_generate_model = "mixtral"
ollama_chat_model = "mixtral"
#ollama_model = "llama2"
default_temperature = 0.7 # as suggested by Mistral API docs https://docs.mistral.ai/api/

# Helper functions



In [36]:
# Helper functions  
def ollama_chat(messages, temperature=default_temperature):
    
    data = {
        'model': ollama_chat_model,
        'messages': messages,   
        'temperature': temperature,
        'stream': False
    }
    
    json_data=json.dumps(data)
    
    response = requests.post(ollama_chat_url, data=json_data, headers={"Content-Type": "application/json"})

    if response.status_code == 200:
        response_json = response.json()
        actual_response = response_json["message"] 
        return actual_response
    else:
        return f"Request failed with status code {response.status_code}"

def ollama_generate(prompt, system="you are a helpfull assitant", temperature=default_temperature):
    
    data = {
        'model': ollama_generate_model,
        'prompt': prompt,
        'system': system,        
        'temperature': temperature,
        'stream': False
    }
    
    json_data=json.dumps(data)
    actual_response="error"
    
    response = requests.post(ollama_generate_url, data=json_data, headers={"Content-Type": "application/json"})
    if response.status_code == 200:
        response_json = response.json()
        actual_response = response_json["response"] 
        return actual_response
    else:
        return f"Request failed with status code {response.status_code}"

    

# Sample usage of ollama_generate

In [37]:
# Sample usage of ollama_generate

prompt="what is the meaning of life?"
system="talk like a pirate"
response=ollama_generate(prompt,system=system)
print(response)
   

 Arr matey, ye be askin' a question that philosophers and seafarers alike have pondered for centuries. As a pirate, I'd say the meaning o' life be about findin' treasure, adventure, and camaraderie on the high seas.

But in all seriousness, the meaning o' life is different for everyone. Some might say it's to seek knowledge and understanding, while others believe it's to help others and make the world a better place. Ultimately, the meaning o' life be what ye make it, so get out there and create yer own adventures, matey!


# Sample usage of ollama_chat

In [10]:
# Sample usage of ollama_chat
messages= [
        {"role": "system", "content": "talk like a surfer dude."},
        {"role": "user", "content": "What is the meaning of life."},
##        {"role": "assistant", "content": "possible responses in the context?"},
##        {"role": "user", "content": "more stuff"},
]
#print(json.dumps(messages,indent=4))

response=ollama_chat(messages)

print(response)
#print(type(response))
#show_response(response)    

{"model":"mixtral","created_at":"2024-01-15T16:56:36.450374Z","message":{"role":"assistant","content":" Dude, you're really going deep with that question! Like, you want to know the meaning of life, ya? Well, I reckon it's all about hanging ten on those gnarly waves of experience, feeling the rush of living in the present moment, and being stoked about the good vibes.\n\nYou see, brah, life ain't always about scoring the perfect barrel or landing that sick trick. Sometimes it's about wiping out, learning from our mistakes, and getting back up to ride again. It's about cherishing those moments with our bros and brahs, creating unforgettable memories, and sharing some aloha spirit.\n\nSo, I'd say the meaning of life is finding your own flow, staying true to yourself, and spreading good vibes all around. Just remember, when in doubt, paddle out! Catch you on the next swell, dude!"},"done":true,"total_duration":5683339500,"load_duration":854833,"prompt_eval_duration":476231000,"eval_count"

# Loading Srt File



In [4]:
json_file_url="http://192.168.1.44:8001/1_1_Planteando_objetivos_politica_economica.json"



In [24]:
import requests
import json



try:
    response = requests.get(json_file_url)
    response.raise_for_status()  # Check for any HTTP errors

    json_data = response.text  # Get the JSON content from the response

    # Parse the JSON data into a Python dictionary
    data_dict = json.loads(json_data)

    # Now you can work with the data_dict as needed
    print(type(data_dict))
   # print(data_dict)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

<class 'list'>


In [25]:
print(data_dict[0])


1
00:00:00,000 --> 00:00:59,820
¿Por qué la inflación y el pleno empleo están en conflicto? ¿Por qué? ¿Qué pensáis? ¿Por qué intentar controlar la inflación nos perjudica la plena ocupación? ¿Sería esto? Y al revés, ¿no? ¿Por qué favorecer la plena ocupación nos va a perjudicar la inflación? Esa es tu pregunta, ¿no? Venga, va, un minuto. A ver. Venga, va. ¿Quién nos dice algo? Vale. Arturo, te lo voy a pasar y os lo vais pasando, ¿vale? No os rompáis que te vale una pasta esto. Venga, Arturo. No, no, ya está, ya está. Porque si tenemos plena ocupación significa que, bueno, si estamos favoreciendo la plena ocupación, significa que no hay problema.


In [26]:
import re

def break_str_fragment(input_string):
    # Split the input string by newlines and strip leading/trailing whitespace
    sections = input_string.strip().split('\n')

    # Error handling for incomplete sections
    if len(sections) < 3:
        return None  # or raise an error

    # Extract the number, ensure it's a valid integer
    try:
        number = int(sections[0])
    except ValueError:
        return None  # or raise an error

    # Extract timestamps, handle potential regex match issues
    timestamps_match = re.findall(r'(\d+:\d+:\d+,\d+) --> (\d+:\d+:\d+,\d+)', sections[1])
    if not timestamps_match:
        return None  # or raise an error

    # Extract and concatenate the text lines
    text = ' '.join(sections[2:])

    # Create a dictionary
    tmp_dict = {
        "number": number,
        "timestamp_start": timestamps_match[0][0],
        "timestamp_end": timestamps_match[0][1],
        "text": text
    }

    return tmp_dict


In [32]:
print(type(data_dict))


<class 'list'>


In [29]:
list_of_unprocessed_chunks = []
for fragment in data_dict:  # assuming data_dict is a list of SRT fragments
    list_of_unprocessed_chunks.append(break_str_fragment(fragment))

In [46]:
print(list_of_unprocessed_chunks[0])


{'number': 1, 'timestamp_start': '00:00:00,000', 'timestamp_end': '00:00:59,820', 'text': '¿Por qué la inflación y el pleno empleo están en conflicto? ¿Por qué? ¿Qué pensáis? ¿Por qué intentar controlar la inflación nos perjudica la plena ocupación? ¿Sería esto? Y al revés, ¿no? ¿Por qué favorecer la plena ocupación nos va a perjudicar la inflación? Esa es tu pregunta, ¿no? Venga, va, un minuto. A ver. Venga, va. ¿Quién nos dice algo? Vale. Arturo, te lo voy a pasar y os lo vais pasando, ¿vale? No os rompáis que te vale una pasta esto. Venga, Arturo. No, no, ya está, ya está. Porque si tenemos plena ocupación significa que, bueno, si estamos favoreciendo la plena ocupación, significa que no hay problema.'}


In [43]:
prompt="""
Take the following transcription of a lecture in spanish, 
remove all the repetitions, interactions with students,and other nonsensical speech,
and output only a well written version of what is being explained. write in spanish.
---Begin text ---
{}
---End text ---
""".format(list_of_unprocessed_chunks[0]['text'])

system="""you are a helpfull asistant who helps in creating lecture notes from lecture transcriptions. 
You output only the reviewed text. You write always in spanish of spain
"""

print(prompt_test)


Take the following transcription of a lecture in spanish, 
remove all the repetitions, interactions with students,and other nonsensical speech,
and output only a well written version of what is being explained. write in spanish.
---Begin text ---
¿Por qué la inflación y el pleno empleo están en conflicto? ¿Por qué? ¿Qué pensáis? ¿Por qué intentar controlar la inflación nos perjudica la plena ocupación? ¿Sería esto? Y al revés, ¿no? ¿Por qué favorecer la plena ocupación nos va a perjudicar la inflación? Esa es tu pregunta, ¿no? Venga, va, un minuto. A ver. Venga, va. ¿Quién nos dice algo? Vale. Arturo, te lo voy a pasar y os lo vais pasando, ¿vale? No os rompáis que te vale una pasta esto. Venga, Arturo. No, no, ya está, ya está. Porque si tenemos plena ocupación significa que, bueno, si estamos favoreciendo la plena ocupación, significa que no hay problema.
---End text ---



In [45]:

response=ollama_generate(prompt_test,system=system)
print(response)

 La plena ocupación y la inflación suelen estar en conflicto. Intentar controlar la inflación puede perjudicarnos en términos de plena ocupación, ya que fomentar el pleno empleo implica que no hay problemas en este ámbito.

En resumen, la relación entre inflación y pleno empleo es inversa: cuando una sube, la otra tiende a bajar, y viceversa. Esto se debe a que, en un contexto de plena ocupación, la demanda de bienes y servicios supera a la oferta, lo que puede desencadenar un aumento de precios (inflación). Por tanto, es complejo lograr simultáneamente el equilibrio entre ambas metas económicas.


In [47]:
import copy

list_of_processed_chunks = []
for unprocessed_chunk in list_of_unprocessed_chunks:
    prompt = f"""
    Take the following transcription of a lecture in Spanish, 
    remove all the repetitions, interactions with students, and other nonsensical speech,
    and output only a well-written version of what is being explained. Write in Spanish.
    ---Begin text ---
    {unprocessed_chunk['text']}
    ---End text ---
    """
    
    processed_chunk = copy.deepcopy(unprocessed_chunk)
    try:
        processed_chunk['text'] = ollama_generate(prompt, system=system)
        print(processed_chunk['text'])  # Uncomment for debugging
    except Exception as e:
        print(f"Error processing chunk: {e}")
        continue

    list_of_processed_chunks.append(processed_chunk)
    

 La plena ocupación y la inflación pueden estar en conflicto. La pregunta es por qué, cuando se promueve el pleno empleo, puede perjudicar el control de la inflación y viceversa. Cuando existe plena ocupación, no hay problemas económicos significativos, lo que sugiere que un mayor nivel de empleo podría influir en el aumento de los precios y, por tanto, en la inflación. Sin embargo, es necesario un análisis más detallado para comprender este posible conflicto entre dos conceptos económicos importantes.
 La política económica mencionada tiene como objetivo proporcionar empleo a un mayor número de personas, lo que resultaría en una amplia producción de bienes y riquezas. No obstante, debido a la gran oferta de estos bienes y riquezas, su valor tendería a disminuir. Además, con más personas capacitadas para brindar los mismos servicios, habría una mayor cantidad de dinero en circulación, lo que llevaría a que la gente deseara adquirir más bienes. Por lo tanto, con más dinero en circulació

In [48]:
import json

# Assuming list_of_processed_chunks is your final data to be saved
data_to_save = list_of_processed_chunks

# Specify the filename
filename = 'processed_chunks.json'

# Writing JSON data
with open(filename, 'w', encoding='utf-8') as file:
    json.dump(data_to_save, file, ensure_ascii=False, indent=4)
